In [16]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from htrace import Parse
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

%config InlineBackend.figure_formats = ['svg']
sns.set_style("whitegrid")

In [17]:
data_path={
    'dfsioe_r': 'data/dfsioe_r/',
    'dfsioe_w': 'data/dfsioe_w/',
    'kmeans': 'data/kmeans/',
    'pagerank': 'data/pagerank/',
    'terasort': 'data/terasort/',
    'wordcount': 'data/wordcount/'
}
samplers = [
#             'a', 
#             'p0.1', 
            'l', 
#             't'
]
zh_samplers = ['全采样','限制采样','令牌桶采样','概率采样(0.1)','概率采样(0.01)']

## 特定函数(次数不多)性能影响

In [24]:
def clean_src_csv(path):
    df = pd.read_csv(path+'/1.csv')
    df.index = df['Unnamed: 0']
    df.drop(['Unnamed: 0'],axis=1, inplace=True)
    df = df.T
#     df =df.drop(['25%','50%'], axis=1)
    df.columns = list(map(lambda x: path.split('/')[2]+'-'+x,['count','mean','std','min','25%','50%','75%','max']))
    return df

In [25]:
def bench_func_display(bench):
    res = []
    for sampler in samplers:
        res.append(clean_src_csv(bench+sampler))
    c = pd.concat(res, axis=1, sort=True)
#     c.columns = zh_samplers
#     c.insert(0, 'time', c['a-count']*c['a-mean'])
    return c[
(c['l-count']>10)
#              (c['l-std']>c['l-mean']+1)
#              (c['p0.1-count']>c['l-count']+1)
    ]

In [30]:
for name, bench in data_path.items():
    print(bench)
    c = bench_func_display(bench)
    tar = pd.read_csv(bench+'l/1s.csv')
    err = []
    x = []
    for i in c.iterrows():
        d = tar[tar['name']==i[0]]
        res = 0
        for j in d.iterrows():
            if j[1]['time'] > 1.5*i[1]['l-50%']:
                res += 1
        err.append((res/i[1]['l-count'])*100)
        x.append(res)
    c['err%'] = err
    c['res'] = x
    c[c['err%']<10].sort_values('err%')

data/dfsioe_r/


,l-count,l-mean,l-std,l-min,l-25%,l-50%,l-75%,l-max,err%,res
DFSInputStream#fetchBlockAt,11.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0
DFSOutputStream#writeChunk,1375.0,0.008727,0.093045,0.0,0.0,0.0,0.0,1.0,0.872727,12
ClientNamenodeProtocol#getFileInfo,81.0,0.567901,0.522931,0.0,0.0,1.0,1.0,2.0,1.234568,1
ClientNamenodeProtocol#getBlockLocations,70.0,0.785714,0.446982,0.0,1.0,1.0,1.0,2.0,1.428571,1
DFSInputStream#readWithStrategy,1928.0,0.062759,0.716550,0.0,0.0,0.0,0.0,25.0,3.734440,72
DFSOutputStream#close,37.0,25.216216,22.456297,6.0,18.0,23.0,24.0,149.0,5.405405,2
DFSOutputStream#flushInternal,37.0,18.756757,20.684677,0.0,14.0,15.0,20.0,137.0,5.405405,2
waitForAckedSeqno,37.0,18.459459,20.524504,0.0,14.0,14.0,20.0,136.0,5.405405,2
DFSInputStream#byteArrayRead,1928.0,0.112033,0.747180,0.0,0.0,0.0,0.0,25.0,8.558091,165


data/dfsioe_w/


,l-count,l-mean,l-std,l-min,l-25%,l-50%,l-75%,l-max,err%,res
ClientNamenodeProtocol#getBlockLocations,38.0,0.631579,0.488852,0.0,0.0,1.0,1.0,1.0,0.000000,0
ClientNamenodeProtocol#getFileInfo,81.0,0.604938,0.491910,0.0,0.0,1.0,1.0,1.0,0.000000,0
DFSOutputStream#writeChunk,19471.0,0.002773,0.052591,0.0,0.0,0.0,0.0,1.0,0.277336,54
getFileInfo,81.0,0.790123,0.585578,0.0,0.0,1.0,1.0,4.0,2.469136,2
getBlockLocations,38.0,1.026316,0.716102,0.0,1.0,1.0,1.0,5.0,2.631579,1
DFSInputStream#fetchBlockAt,34.0,0.029412,0.171499,0.0,0.0,0.0,0.0,1.0,2.941176,1
DFSInputStream#openInfo,37.0,1.216216,1.204970,0.0,1.0,1.0,1.0,7.0,5.405405,2
DFSInputStream#readWithStrategy,826.0,0.105327,0.863984,0.0,0.0,0.0,0.0,23.0,6.053269,50
DFSInputStream#byteArrayRead,826.0,0.122276,0.872926,0.0,0.0,0.0,0.0,23.0,7.627119,63
newDFSInputStream,37.0,1.432432,2.062099,0.0,1.0,1.0,1.0,12.0,8.108108,3


data/kmeans/


...

,l-count,l-mean,l-std,l-min,l-25%,l-50%,l-75%,l-max,err%,res
DFSOutputStream#writeChunk,14841.0,0.001550,0.039338,0.0,0.0,0.0,0.0,1.0,0.154976,23
ClientNamenodeProtocol#getFileInfo,1024.0,0.743164,3.445633,0.0,0.0,1.0,1.0,60.0,1.171875,12
DFSInputStream#fetchBlockAt,49.0,0.020408,0.142857,0.0,0.0,0.0,0.0,1.0,2.040816,1
getFileInfo,1024.0,0.891602,3.978801,0.0,0.0,1.0,1.0,88.0,2.148438,22
ClientNamenodeProtocol#rename,483.0,5.086957,2.106278,3.0,4.0,5.0,5.0,23.0,3.105590,15
DFSClient#rename,483.0,5.171843,2.107454,3.0,4.0,5.0,5.0,23.0,3.933747,19
DFSInputStream#readWithStrategy,1713.0,0.117922,0.789157,0.0,0.0,0.0,0.0,24.0,5.662580,97
listPaths,91.0,1.340659,3.565949,0.0,0.5,1.0,1.0,33.0,8.791209,8
DFSInputStream#byteArrayRead,1713.0,0.149445,0.803665,0.0,0.0,0.0,0.0,24.0,8.814945,151
ClientNamenodeProtocol#complete,247.0,8.105263,12.291061,3.0,5.0,6.0,7.0,104.0,9.311741,23


data/terasort/


,l-count,l-mean,l-std,l-min,l-25%,l-50%,l-75%,l-max,err%,res
DFSOutputStream#writeChunk,6372.0,0.003453,0.058662,0.0,0.0,0.0,0.0,1.0,0.345261,22
DFSInputStream#readWithStrategy,4638.0,0.026089,1.312819,0.0,0.0,0.0,0.0,89.0,0.539025,25
ClientNamenodeProtocol#getFileInfo,213.0,0.845070,3.782960,0.0,0.0,1.0,1.0,55.0,2.347418,5
getFileInfo,213.0,1.197183,6.419942,0.0,0.0,1.0,1.0,94.0,3.755869,8
DFSInputStream#byteArrayRead,4638.0,0.066192,1.326614,0.0,0.0,0.0,0.0,89.0,4.549375,211
DFSOutputStream#flushInternal,42.0,4.214286,9.981778,0.0,2.0,3.0,3.0,67.0,4.761905,2
ClientNamenodeProtocol#fsync,40.0,5.250000,3.069703,3.0,4.0,5.0,5.0,23.0,5.000000,2
DataStreamer#writeTo,507.0,0.601578,0.585596,0.0,0.0,1.0,1.0,2.0,5.128205,26
DFSOutputStream#close,42.0,9.309524,12.487043,6.0,6.0,6.0,6.0,78.0,9.523810,4


data/wordcount/


,l-count,l-mean,l-std,l-min,l-25%,l-50%,l-75%,l-max,err%,res
DFSInputStream#readWithStrategy,4457.0,0.034777,1.561401,0.0,0.0,0.0,0.0,104.0,1.144267,51
ClientNamenodeProtocol#getFileInfo,204.0,0.823529,4.196907,0.0,0.0,1.0,1.0,60.0,1.960784,4
ClientNamenodeProtocol#getListing,42.0,0.547619,0.550050,0.0,0.0,1.0,1.0,2.0,2.380952,1
DFSOutputStream#writeChunk,77.0,0.025974,0.160101,0.0,0.0,0.0,0.0,1.0,2.597403,2
ClientNamenodeProtocol#create,41.0,6.926829,4.966841,2.0,6.0,6.0,7.0,36.0,4.878049,2
DFSOutputStream#close,41.0,22.975610,11.913202,5.0,17.0,23.0,23.0,88.0,4.878049,2
DFSOutputStream#flushInternal,41.0,18.317073,10.527201,0.0,14.0,19.0,20.0,75.0,4.878049,2
DistributedFileSystem#create,41.0,8.048780,6.917193,2.0,6.0,7.0,8.0,39.0,4.878049,2
newStreamForCreate,41.0,7.804878,6.100080,2.0,6.0,7.0,8.0,36.0,4.878049,2
waitForAckedSeqno,41.0,18.121951,10.271794,0.0,14.0,18.0,20.0,73.0,4.878049,2
